# Estimaciones del consumo de sustancias en México

Apartir de los datos de la Encuesta Nacional de Consumo de Drogas, Alcohol y Tabaco, ENCODAT 2016-2017.


### Preparativos

In [1]:
# ---------- # Importación de bibliotecas # ------------------ #
import os
import sys
import pandas as pd

# ---------- # Importación de funciones # ------------------ #
from modulos.func_analisis import *
from modulos.func_transformacion import *

In [2]:
# ------------------ # Carga de las bases de datos # ------------------ #
h=pd.read_csv(os.path.join('datos','originales','ENCODAT_2016_2017_Hogar','ENCODAT_2016_2017_Hogar.csv'), sep=';', low_memory=False)
bb=pd.read_csv(os.path.join('datos','originales','ENCODAT_2016_2017_Individual','ENCODAT_2016_2017_Individual.csv'), sep=';', low_memory=False)

In [3]:
# ---------- # Adecuaciones a las bases de datos # ------------------ #

# Homologación y corrección del ponderador para que sea del tipo numérico
bb.insert(0,'factor_exp',bb['ponde_ss'].str.partition(',')[0].astype(int))
h.insert(0,'factor_exp',h['ponde_hh'].str.partition(',')[0].astype(int))

# Corrección de codigos de las unidades primarias de muestreo para que sean sólo numéricos
h.insert(0,'upm',h['code_upm'].str.replace('_','').astype(int))

# Inclusión id_hogar en el cuestionario individual (bb)
bb.insert(0,'id_hogar',bb['id_pers'].str[:20])

# Inclusión de la unidad primaria de muestreo (upm) y del estrato (estrato) en el cuestionario individual mediante id_hogar
b=bb.merge(h[['id_hogar','estrato','upm','code_upm','est_var']],on='id_hogar',how='left')

# Creación de grupos etarios 
#(los grupos etarios se pueden cambiar si así se desea) 
grupos_etarios=['12-17', '18-34', '35-59', '60-75']
divisiones=[int(x[:2])-1 for x in grupos_etarios]+[int(grupos_etarios[-1][-2:])]
b.insert(len(b.columns),'grupo_etario',pd.cut(list(b['ds3']), bins=divisiones, labels=grupos_etarios))

# Cambio de nombre y valores de las columnas entidad y sexo
b.rename(columns={'entidad':'cve_ent','desc_ent':'nom_ent','ds2':'sexo'}, inplace=True)
valores_sexo = {1:'Hombre', 2:'Mujer'}
b['sexo'] = b['sexo'].map(valores_sexo).fillna(b['sexo'])
b['nom_ent']=b['nom_ent'].str.strip().apply(arreglar_palabras)

#Homologación de variables a tipo numérico
lista_vars=['al4','tb08','di1a','di1b','di1c','di1d','di1e','di1f','di1g','di1h','di1i']
for v in lista_vars:
    b[v]=b[v].replace(' ',0)
    b[v]=b[v].astype(int)

### Estimaciones sobre consumo de alcohol

El consumo de alcohol se estimó a partir de las respuestas afirmativas a la pregunta con clave `al4`:

---
**En los últimos 12 meses, ¿tomó alguna bebida que contenga alcohol? (cerveza, pulque, vino, brandy, whisky, ron, tequila, culers, Niuwmix, Cubaraima, etc.)**
1. Sí
2. No
---

Es decir, para estimar el consumo de alcohol, en la base de datos se contabilizaron los registros con valor `1` en la variable `al4`. 

In [4]:
# ---------- # Alcohol # ------------------ #

# Estimación nacional del desagregada por sexo y grupo etario
al_nac = tabulados_prevalencias(b,['al4'],{1:1,2:0})

# Estimación estatal del desagregada por sexo y grupo etario
al_edo= tabulados_prevalencias(b,['al4'],{1:1,2:0},['nom_ent','cve_ent'])

In [18]:
# Tabulados
al_nac.head()

,sexo,grupo_etario,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
1,Mujeres y hombres,Población de 12-75 años,47.793007,46.844539,48.741475,40735726,0.483921,1.012535
0,Hombre,Población de 12-75 años,58.534996,57.225000,59.844993,24091060,0.668378,1.141843
1,Mujer,Población de 12-75 años,37.762723,36.699105,38.826342,16644666,0.542672,1.437059
4,Hombre,12-17,25.658860,23.877599,27.440122,1858919,0.908823,3.541948
5,Hombre,18-34,70.513183,68.439228,72.587137,11587829,1.058159,1.500655


In [17]:
al_edo.head()

,sexo,grupo_etario,nom_ent,cve_ent,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
0,Mujeres y hombres,Población de 12-75 años,01 Aguascalientes,1,57.124669,53.897940,60.351398,533938,1.646321,2.881978
1,Mujeres y hombres,Población de 12-75 años,02 Baja California,2,43.339070,40.130882,46.547259,1056597,1.636861,3.776871
2,Mujeres y hombres,Población de 12-75 años,03 Baja California Sur,3,49.717380,46.044295,53.390465,263962,1.874057,3.769421
3,Mujeres y hombres,Población de 12-75 años,04 Campeche,4,44.855680,41.256993,48.454367,294661,1.836099,4.093347
4,Mujeres y hombres,Población de 12-75 años,05 Coahuila de Zaragoza,5,47.860241,44.212105,51.508378,1020936,1.861328,3.889091


In [6]:
# Exportación de los tabulados
al_nac.to_csv(os.path.join('datos', 'procesados', 'prevalencia_consumo_alcohol_nal_2016-2017.csv'), index = False)
al_edo.to_csv(os.path.join('datos', 'procesados','prevalencia_consumo_alcohol_edo_2016-2017.csv'), index= False)

### Estimaciones sobre consumo de tabaco

El consumo de tabaco se estimó a partir de las personas que respondieron haber fumado tabaco en el último año, en la pregunta `tb08`:

---
**¿Cuándo fue la última vez que se fumó un cigarro?**
1. En los últimos 30 días

2. Más de 1 mes pero menos de 6 meses

3. Hace 6 meses o más pero menos de 1 años

4. Hace 1 año o más pero menos de 3 años

5. Hace más de 3 años
---

Por lo tanto, para estimar el consumo de tabaco, en la base de datos se contabilizaron los registros con valor `1`, `2` o `3` para la variable `tb08`:

In [7]:
# ---------- # Tabaco # ------------------ #

# Estimación nacional del desagregada por sexo y grupo etario
tb_nac = tabulados_prevalencias(b,['tb08'],{1:1,2:1,3:1,4:0,5:0})

# Estimación estatal del desagregada por sexo y grupo etario
tb_edo = tabulados_prevalencias(b,['tb08'],{2:1,3:1,4:0,5:0},['nom_ent','cve_ent'],estrato='estrato')

In [16]:
# Tabulados
tb_nac.head()

,sexo,grupo_etario,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
1,Mujeres y hombres,Población de 12-75 años,16.779887,16.063334,17.496441,14302111,0.365595,2.178770
0,Hombre,Población de 12-75 años,25.911535,24.754167,27.068903,10664327,0.590505,2.278926
1,Mujer,Población de 12-75 años,8.253252,7.630810,8.875695,3637784,0.317578,3.847919
4,Hombre,12-17,6.369720,5.468600,7.270841,461470,0.459764,7.217958
5,Hombre,18-34,33.762901,31.760164,35.765639,5548448,1.021824,3.026469


In [13]:
tb_edo.head()

,sexo,grupo_etario,nom_ent,cve_ent,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
0,Mujeres y hombres,Población de 12-75 años,01 Aguascalientes,1,21.934676,19.375618,24.493733,205021,1.305666,5.952518
1,Mujeres y hombres,Población de 12-75 años,02 Baja California,2,16.473652,14.078488,18.868815,401624,1.222045,7.418178
2,Mujeres y hombres,Población de 12-75 años,03 Baja California Sur,3,16.322456,13.989581,18.655331,86660,1.190264,7.292188
3,Mujeres y hombres,Población de 12-75 años,04 Campeche,4,12.452562,10.009240,14.895884,81802,1.246616,10.010918
4,Mujeres y hombres,Población de 12-75 años,05 Coahuila de Zaragoza,5,22.800201,19.044099,26.556304,486365,1.916414,8.405250


In [9]:
# Exportación de los tabulados
tb_nac.to_csv(os.path.join('datos', 'procesados', 'prevalencia_consumo_tabaco_nal_2016-2017.csv'), index= False)
tb_edo.to_csv(os.path.join('datos', 'procesados', 'prevalencia_consumo_tabaco_edo_2016-2017.csv'), index= False)

### Estimaciones sobre consumo de drogas

El consumo de drogas se estimó a partir de varias preguntas:

---
`di1a` ***Continuando con el uso de sustancias. Me podría decir si ha tomado, usado, probado MARIGUANA. también llamada 'hashish', 'mota', 'café', 'yerba', etc. para drogarse.***
1. Sí
2. No
9) No sabe/No contesta
---

---
`di1b` ***Me podría decir si ha tomado, usado, probado COCAÍNA, incluyendo todas las diferentes formas como polvo, pasta base y pasta de coca,  también llamada 'perico', 'nieve', 'grapa', 'coca', etc.***
1. Sí
2. No
9) No sabe/No contesta
---

---
`di1c` ***Me podría decir si ha tomado, usado, probado CRACK, también llamado 'piedra'.***
1. Sí
2. No
9) No sabe/No contesta
---

---
`di1d` ***Me podría decir si ha tomado, usado, probado  ALUCINÓGENOS como hongos, peyote, mezcalina, LSD conocido como 'trip' o 'viaje', PCP, también llamados 'ácidos', 'champiñones', 'aceites', etc.***
1. Sí
2. No
9) No sabe/No contesta
---

---
`di1e` ***Me podría decir si ha tomado, usado, probado INHALABLES: como thiner, PVC, cemento, resistol, pegamento, pintura, gasolina, activo, sprays, llamados 'chemos', 'memos', 'monas', 'solventes', etc. para drogarse.***
1. Sí
2. No
9) No sabe/No contesta
---

---
`di1f` ***Me podría decir si ha tomado, usado, probado HEROÍNA, opio, también llamada 'arpón', 'ficción', 'chiva', 'la H', 'speed ball'.***
1. Sí
2. No
9) No sabe/No contesta
---

---
`di1g` ***Me podría decir si ha tomado, usado, probado ESTIMULANTES TIPO ANFETAMÍNICO, droga de diseño, éxtasis, conocido también como 'tachas', MDMA, cristal.***
1. Sí
2. No
9) No sabe/No contesta
---

---
`di1h` ***Me podría decir si ha tomado, usado, probado OTRAS DROGAS como: ketamina, (special K), GHB, también conocido como éxtasis líquido.***
1. Sí
2. No
9) No sabe/No contesta
---

---
`di1i` ***Me podría decir si ha tomado, usado, probado Mariguana Sintética (Spice, K2, Yucatán Fire)***
1. Sí
2. No
9) No sabe/No contesta
---

En este caso, para estimar el consumo de drogas se consideraron los registros que respondieron afirmativamente a alguna de las preguntas, es decir, registros con valor `1` en `di1a`, `di1b`, `di1c`, `di1d`, `di1e`, `di1f`, `di1g`, `di1h` o `di1i`:

In [10]:
# ---------- # Drogas # ------------------ #

# Estimación nacional del desagregada por sexo y grupo etario
dr_nac = tabulados_prevalencias(b,['di1a','di1b','di1c','di1d','di1e','di1f','di1g','di1h','di1i'], {1:1,2:0,9:0})

# Estimación estatal del desagregada por sexo y grupo etario
dr_edo= tabulados_prevalencias(b,['di1a','di1b','di1c','di1d','di1e','di1f','di1g','di1h','di1i'], {1:1,2:0,9:0},['nom_ent','cve_ent'])

In [15]:
# tabulados
dr_nac.head()

,sexo,grupo_etario,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
1,Mujeres y hombres,Población de 12-75 años,10.082045,9.559705,10.604384,8593295,0.266505,2.643360
0,Hombre,Población de 12-75 años,16.100935,15.177134,17.024736,6626610,0.471336,2.927381
1,Mujer,Población de 12-75 años,4.461933,4.018501,4.905364,1966685,0.226245,5.070556
4,Hombre,12-17,6.912555,5.809632,8.015478,500797,0.562726,8.140641
5,Hombre,18-34,22.720939,20.984347,24.457530,3733860,0.886032,3.899629


In [14]:
dr_edo.head()

,sexo,grupo_etario,nom_ent,cve_ent,estimacion,ic_inf,ic_sup,poblacion,error_std,cv
0,Mujeres y hombres,Población de 12-75 años,01 Aguascalientes,1,11.168635,8.822631,13.514638,104392,1.196963,10.717179
1,Mujeres y hombres,Población de 12-75 años,02 Baja California,2,13.673913,11.549993,15.797834,333367,1.083653,7.924965
2,Mujeres y hombres,Población de 12-75 años,03 Baja California Sur,3,10.991760,8.578831,13.404689,58358,1.231109,11.200289
3,Mujeres y hombres,Población de 12-75 años,04 Campeche,4,11.568726,8.868487,14.268965,75996,1.377698,11.908816
4,Mujeres y hombres,Población de 12-75 años,05 Coahuila de Zaragoza,5,11.089740,8.443673,13.735806,236562,1.350059,12.173943


In [12]:
# Exportación de los tabulados
dr_edo.to_csv(os.path.join('datos', 'procesados','prevalencia_consumo_drogas_edo_2016-2017.csv'), index= False)
dr_nac.to_csv(os.path.join('datos', 'procesados', 'prevalencia_consumo_drogas_nal_2016-2017.csv'), index = False)